# Hubble deep space sonification

Example inspired from https://www.nasa.gov/content/explore-from-space-to-sound

In [ ]:
import ipytone
import skimage
import numpy as np
import ipywidgets as widgets
import bqplot
import bqplot.pyplot as bqp

In [ ]:
t = ipytone.transport

In [ ]:
image = skimage.data.hubble_deep_field()

Create a fixed number of oscillators at evenly spaced frequencies (within a given range). Each oscillator is connected to its own gain node, which will be controlled from the image data (y-axis).

Connect all gain nodes to a `Panner` node so that we'll hear the sound going from the left to the right stereo channel while scanning and playing the image from left to right. 

In [ ]:
n_oscillators = 40
freq_min = 50
freq_max = 1000

freqs = np.linspace(freq_min, freq_max, n_oscillators)

oscillators = []
gains = []

pan = ipytone.Panner().to_destination()

for freq in freqs:
    osc = ipytone.Oscillator(frequency=freq, volume=-15)
    gain = ipytone.Gain(gain=0)
    osc.chain(gain, pan)
    
    oscillators.append(osc)
    gains.append(gain)


Resize the image so that the number of rows matches the number of oscillators created above, and extract the instensity (which will be used directly to control the gain nodes).

In [ ]:
# resize
new_shape = [n_oscillators, image.shape[1]]
resized = skimage.transform.resize(image, new_shape)

# get intensity
intensity = skimage.color.rgb2gray(resized)

# remove noise
intensity = np.where(intensity > 0.1, intensity, 0)

# will start and stop with all gains set to zero
intensity = np.pad(intensity, pad_width=((0, 0), (1, 1)))

Generate the automation curves (gains, panning, etc.) for a given time range.

In [ ]:
duration = 30

def osc_start_stop(time):
    for osc in oscillators:
        osc.start(time).stop(time + duration)

def gain_automation(time):
    for i in range(1, n_oscillators + 1):
        gains[i - 1].gain.set_value_curve_at_time(intensity[-i], time, duration)

def pan_automation(time):
    pan.pan.set_value_at_time(-1, time)
    pan.pan.linear_ramp_to_value_at_time(1, time + duration)

osc_eid = t.schedule(osc_start_stop, 0)
gain_eid = t.schedule(gain_automation, 0)
pan_eid = t.schedule(pan_automation, 0)

Create the interactive figure with `bqplot`.

In [ ]:
skimage.io.imsave("temp.png", image)

with open("temp.png", "rb") as f:
    raw_image = f.read()

ipyimage = widgets.Image(value=raw_image, format="png")

In [ ]:
scales = {
    "x": bqplot.LinearScale(min=0, max=duration),
    "y": bqplot.LinearScale(min=freq_min, max=freq_max)
}

bqimage = bqplot.Image(
    image=ipyimage,
    scales=scales,
    x=(0, 30),
    y=(freq_min, freq_max)
)

lines = bqplot.Lines(
    x=np.zeros(n_oscillators) + 0.25,
    y=freqs,
    scales=scales,
)
lines.colors = ["white"]

vline = bqp.vline(0, scales=scales)
vline.colors = ["white"]

fig = bqplot.Figure(marks=[bqimage, lines, vline])
fig.layout.width = "500px"
fig.layout.height = "500px"

Interactive lines showing the current time position on the figure as well as the current gain for each frequency.

In [ ]:
def move_lines(change):
    time = change["new"]
    vline.x = [time, time]
    
    idx = int(intensity.shape[1] / duration * time)
    lines.x = time + 0.25 + intensity[::-1, idx] * 5


In [ ]:
t.schedule_observe(
    move_lines, update_interval=0.1, name="seconds", transport=True
)

In [ ]:
fig

Let's see and hear the result!

In [ ]:
t.start().stop(f"+{duration}")

Clean-up

In [ ]:
t.schedule_unobserve(move_lines)
t.cancel()
t.stop()

In [ ]:
for osc in oscillators:
    osc.stop()

In [ ]:
for osc in oscillators:
    osc.close()

for gain in gains:
    gain.close()
    
pan.close()